# 案例 1.4: 数据清洗与评估诊断

本案例演示了平台的数据处理能力。我们将创建一个时间序列，手动为其添加缺陷（缺失值和异常值），然后使用平台的工具进行清洗，并最终评估清洗的效果。

### 1. 导入库并加载配置

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 导入平台模块
from swp.data_processing.cleaner import fill_missing_with_interpolation
from swp.data_processing.evaluator import calculate_rmse, calculate_nse, calculate_kge

CONFIG_PATH = 'examples/data_processing_and_evaluation.json'

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 生成和“污染”数据

我们首先根据配置生成一套“完美”的基准数据，然后根据配置中的定义，向其中添加缺失值和异常值，以模拟真实世界中不完美的数据。

In [ ]:
def generate_data(cfg):
    gen_cfg = cfg['data_generation']
    imp_cfg = cfg['imperfections']
    
    # 生成基准数据
    true_values = pd.Series(
        np.sin(np.linspace(0, gen_cfg['sin_wave_end'], gen_cfg['num_points'])) * gen_cfg['sin_wave_multiplier'] + gen_cfg['base_value']
    )

    # 复制一份用于“污染”
    imperfect_values = true_values.copy()

    # 添加缺失值
    imperfect_values.iloc[imp_cfg['missing_indices']] = np.nan

    # 添加异常值
    for outlier in imp_cfg['outliers']:
        if outlier.get('operation') == 'multiply':
            imperfect_values.iloc[outlier['index']] *= outlier['value']
        else:
            imperfect_values.iloc[outlier['index']] = outlier['value']
            
    print(f"生成了 {gen_cfg['num_points']} 个数据点。")
    print(f"原始数据有 {imperfect_values.isna().sum()} 个缺失值和 {len(imp_cfg['outliers'])} 个异常值。\n")
    
    return true_values, imperfect_values

true_data, imperfect_data = generate_data(config)

### 3. 清洗数据并进行评估

我们使用平台的 `fill_missing_with_interpolation` 函数来填充缺失值，然后将清洗前后的数据与“完美”数据进行比较，以量化清洗效果和异常值的影响。

In [ ]:
# 使用线性插值填充缺失值
cleaned_data = fill_missing_with_interpolation(imperfect_data.copy())
print("步骤 1: 使用插值法填充缺失值完成。")
print(f"数据现在有 {cleaned_data.isna().sum()} 个缺失值。\n")

# 评估 '不完美' 数据作为基线
print("--- 评估不完美数据 (含异常值, 为计算指标已去除NaN) ---")
imperfect_subset = imperfect_data.dropna()
true_subset_for_imperfect = true_data[imperfect_subset.index]
rmse_before = calculate_rmse(true_subset_for_imperfect, imperfect_subset)
nse_before = calculate_nse(true_subset_for_imperfect, imperfect_subset)
kge_before = calculate_kge(true_subset_for_imperfect, imperfect_subset)
print(f"基线 RMSE: {rmse_before:.4f}")
print(f"基线 NSE: {nse_before:.4f}")
print(f"基线 KGE: {kge_before:.4f}\n")

# 评估清洗后的数据
print("--- 评估清洗后数据 ---")
rmse_after = calculate_rmse(true_data, cleaned_data)
nse_after = calculate_nse(true_data, cleaned_data)
kge_after = calculate_kge(true_data, cleaned_data)

print(f"清洗后 RMSE: {rmse_after:.4f}")
print(f"清洗后 NSE: {nse_after:.4f} (越接近1越好)")
print(f"清洗后 KGE: {kge_after:.4f} (越接近1越好)\n")
print("结论: 插值法修复了缺失值，但异常值仍然严重影响了性能指标（例如高RMSE，低NSE/KGE）。")

### 4. 可视化对比

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(true_data, 'g-', label='完美数据 (Ground Truth)', linewidth=2)
plt.plot(imperfect_data, 'ro', label='不完美数据 (Imperfect)', markersize=5, alpha=0.6)
plt.plot(cleaned_data, 'b--', label='清洗后数据 (Cleaned)', linewidth=2)
plt.title('数据清洗效果对比')
plt.xlabel('数据点索引')
plt.ylabel('数值')
plt.legend()
plt.grid(True)
plt.show()